In [1]:
import requests
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

d:\APPS\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
books = pd.read_csv("books.csv")
len(books)

16599

In [3]:
def get_value(data, keys, default=None):
    """
    Safely retrieves a nested value from a dictionary.
    
    Args:
        data (dict): The dictionary to retrieve the value from.
        keys (list): List of keys representing the nested path to the value.
        default: The default value to return if the key path doesn't exist.
        
    Returns:
        The retrieved value or the default value.
    """
    try:
        for key in keys:
            data = data[key]
        return data
    except (KeyError, IndexError, TypeError):
        return default


results = []
for index, row in tqdm(books.iterrows(), total=len(books)):
        
    isbn = str(row["ISBN"])
    book_id = row["book_id"]
    if len(isbn) > 1:
        url = "https://www.googleapis.com/books/v1/volumes?q=isbn:" + isbn
        response = requests.get(url)
        data = response.json()

        for item in data.get("items", []):
            result = {
                "ISBN": isbn,
                "book_id": book_id,
                "title": get_value(item, ["volumeInfo", "title"]),
                "subtitle": get_value(item, ["volumeInfo", "subtitle"]),
                "authors": get_value(item, ["volumeInfo", "authors"], default=[]),
                "publisher": get_value(item, ["volumeInfo", "publisher"]),
                "published_date": get_value(item, ["volumeInfo", "publishedDate"]),
                "description": get_value(item, ["volumeInfo", "description"]),
                "categories": get_value(item, ["volumeInfo", "categories"], default=[]),
                "language": get_value(item, ["volumeInfo", "language"]),
            }
            results.append(result)
    else:
        results.append({
            "ISBN": isbn,
            "book_id": book_id,
            "title": None,
            "subtitle": None,
            "authors": [],
            "publisher": None,
            "published_date": None,
            "description": None,
            "categories": [],
            "language": None,
        })

100%|██████████| 16599/16599 [2:41:33<00:00,  1.71it/s]  


In [4]:
results = pd.DataFrame(results)
results.to_csv("books_augmented.csv", index=False, encoding="utf-8")